# Clustering Crypto

In [142]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [143]:
# Load the crypto_data.csv dataset.
file = "crypto_data.csv"
crypto_data = pd.read_csv(file, index_col=0)
crypto_df = pd.DataFrame(crypto_data)
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [144]:
# Keep all the cryptocurrencies that are being traded.
# new = df_crypto_data[df_crypto_data['IsTrading']== 'True']
# new
crypto_df = crypto_df[crypto_df["IsTrading"] == True]
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [145]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df['Algorithm'].isnull().values.any()


False

In [146]:
crypto_df = crypto_df[crypto_df.Algorithm.isna() == False]
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [147]:
# Remove the "IsTrading" column. 

new_crypto_df = crypto_df.drop(["IsTrading"],axis =1)
new_crypto_df 

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [148]:
# Remove rows that have at least 1 null value.


In [149]:
for column in new_crypto_df.columns:
    print(f"Column {column} has {new_crypto_df[column].isnull().sum()}null values")

Column CoinName has 0null values
Column Algorithm has 0null values
Column ProofType has 0null values
Column TotalCoinsMined has 459null values
Column TotalCoinSupply has 0null values


In [150]:
# Keep the rows where coins are mined.

new_crypto_df = new_crypto_df.dropna(subset= ['TotalCoinsMined'])

new_crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [151]:
for column in new_crypto_df.columns:
    print(f"Column {column} has {new_crypto_df[column].isnull().sum()}null values")

Column CoinName has 0null values
Column Algorithm has 0null values
Column ProofType has 0null values
Column TotalCoinsMined has 0null values
Column TotalCoinSupply has 0null values


In [152]:
# Keep the rows where coins are mined.
new_crypto_df=new_crypto_df[new_crypto_df["TotalCoinsMined"] > 0]
new_crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [153]:
# Create a new DataFrame that holds only the cryptocurrencies names.

new_crypto_df2 = new_crypto_df['CoinName'].to_frame()

new_crypto_df2.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [154]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorith
new_crypto_df =new_crypto_df.drop(['CoinName'],axis =1)
new_crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [155]:
# Use get_dummies() to create variables for text features.
new_crypto_df_dummies = pd.get_dummies(new_crypto_df, columns=['Algorithm','ProofType'])
new_crypto_df_dummies

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [156]:
# Standardize the data with StandardScaler().
crypto_scaled = StandardScaler().fit_transform(new_crypto_df_dummies)
print(crypto_scaled[0:5])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [157]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)
crypto_pca

array([[-0.33166068,  1.00099589, -0.54509805],
       [-0.31502504,  1.00116053, -0.54542907],
       [ 2.30495312,  1.65133317, -0.60616398],
       ...,
       [ 0.32143172, -2.32077099,  0.4281876 ],
       [-0.13958558, -1.89908457,  0.29312739],
       [-0.28321955,  0.78762194, -0.26237611]])

In [158]:
# Create a DataFrame with the three principal components.
df_crypto_pca = pd.DataFrame( data= crypto_pca, columns =["PC 1","PC 2","PC 3"], index=new_crypto_df2.index)
df_crypto_pca

,PC 1,PC 2,PC 3
42,-0.331661,1.000996,-0.545098
404,-0.315025,1.001161,-0.545429
1337,2.304953,1.651333,-0.606164
BTC,-0.140003,-1.332018,0.188597
ETH,-0.156071,-2.034058,0.391684
...,...,...,...
ZEPH,2.498872,0.761853,-0.207611
GAP,-0.329710,1.000881,-0.545106
BDX,0.321432,-2.320771,0.428188
ZEN,-0.139586,-1.899085,0.293127


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [159]:
# Create an elbow curve to find the best value for K.
# YOUR CODE HERE
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_crypto_pca)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

C:\Users\ramak\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [206]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(df_crypto_pca) 

# Predict clusters
predictions = model.predict(df_crypto_pca)
predictions

array([0, 0, 0, 3, 3, 3, 0, 3, 3, 3, 0, 3, 0, 0, 3, 0, 3, 3, 0, 0, 3, 3,
       3, 3, 3, 0, 3, 3, 3, 0, 3, 0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0,
       3, 3, 3, 3, 3, 0, 0, 3, 0, 3, 3, 3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 3,
       3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 3, 0, 3, 0, 0, 3, 3, 3, 3, 0,
       0, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3, 3, 0, 0, 3, 0, 0, 3, 0, 3, 0, 3,
       0, 3, 0, 0, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 0,
       3, 0, 3, 3, 0, 3, 0, 3, 0, 0, 3, 3, 0, 3, 3, 0, 0, 3, 0, 3, 0, 0,
       0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 3, 0, 3, 0, 0, 3, 0, 3, 0, 0, 3, 0, 3, 0, 3, 0, 3, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 3, 0, 3, 0,
       0, 3, 0, 3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0, 3, 0, 3, 0, 0, 0, 0, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 0, 0, 3,
       0, 3, 3, 3, 0, 3, 0, 3, 3, 3, 0, 3, 0, 3, 0,

In [161]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([new_crypto_df, df_crypto_pca], axis=1, join='inner')
clustered_df
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 

Coinname =crypto_df['CoinName']
clustered_df = clustered_df.join(Coinname)
clustered_df


#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# # YOUR CODE HERE
clustered_df["class"] = model.labels_
clustered_df.head()
# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.331661,1.000996,-0.545098,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.315025,1.001161,-0.545429,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.304953,1.651333,-0.606164,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.140003,-1.332018,0.188597,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.156071,-2.034058,0.391684,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.163339,-1.154249,-0.011432,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.404241,1.258949,-0.481188,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.156439,-2.258020,0.362741,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.154516,-2.034158,0.391680,Ethereum Classic,3
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.139585,-1.899085,0.293127,ZCash,3


In [162]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    hover_name="CoinName",
    hover_data=["Algorithm"],
    color="class",
    symbol="class",
    width=800
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [164]:
# Create a table with tradable cryptocurrencies.
from bokeh.sampledata.autompg import autompg_clean as df
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType','TotalCoinSupply','TotalCoinsMined','class'], sortable=True, selectable=True)
# clustered_df.head()

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [167]:
# Print the total number of tradable cryptocurrencies.
clustered_df['CoinName'].count()

532

In [169]:
print('There are 532 tradable cryptocurrencies')

There are 532 tradable cryptocurrencies


In [190]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
from sklearn.preprocessing import MinMaxScaler
data = clustered_df[['TotalCoinSupply','TotalCoinsMined']]
scaler = MinMaxScaler()
print(scaler.fit(data))
# print(scaler.data_max_)
print(scaler.fit_transform(data))
clustered_df3 = scaler.fit_transform(data)

MinMaxScaler()
[[4.20000000e-11 0.00000000e+00]
 [5.32000000e-04 1.06585544e-03]
 [3.14159265e-01 2.95755135e-02]
 ...
 [1.40022261e-03 9.90135079e-04]
 [2.10000000e-05 7.37028150e-06]
 [1.00000000e-06 1.29582282e-07]]


In [191]:
clustered_df3

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [201]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE

new_clustered_df = pd.DataFrame(data =clustered_df3, columns = ['TotalCoinSupply','TotalCoinsMined'],index=new_crypto_df2.index)
new_clustered_df
# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
Coinname = clustered_df['CoinName']
new_clustered_df = new_clustered_df.join(Coinname)
new_clustered_df

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE
class1 =clustered_df["class"]
new_clustered_df =new_clustered_df.join(class1)
new_clustered_df.head(10)
#plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,3
ETH,0.000000e+00,0.000109,Ethereum,3
LTC,8.400000e-05,0.000064,Litecoin,3
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,3
ETC,2.100000e-04,0.000115,Ethereum Classic,3
ZEC,2.100000e-05,0.000007,ZCash,3


In [202]:
import hvplot
import hvplot.pandas

In [203]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
new_clustered_df.hvplot.scatter(
x="TotalCoinsMined",
y="TotalCoinSupply",
hover_cols=["CoinName"],
by="class",
)

:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)